In [1]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LinearRegression

Создадим сэмпл сотрудников и их зарплат

In [2]:
n_samples = 500

# возраст сотрудника от 18 до 60
age_worker = np.random.choice(43, n_samples) + 18
# количество проектов, которые сотрудник выполняет в месяц
projects_month = np.random.choice(5, n_samples) + 1
# количество часов, которые сотрудник в среднем тратит на каждый проект
hours_project = np.random.choice(30, n_samples) + 10
# количесто больничных в меясц
sick_days = np.random.choice(7, n_samples) + 1

# зарплата сотрудников
pay = projects_month * hours_project * 5 + sick_days * 2.5

data = pd.DataFrame({'age_worker': age_worker, 'projects_month': projects_month, 
                     'hours_project': hours_project, 'sick_days': sick_days,
                     'pay': pay})
data.head(5)

,age_worker,projects_month,hours_project,sick_days,pay
0,60,4,23,7,477.5
1,32,1,23,6,130.0
2,29,2,38,2,385.0
3,53,4,32,3,647.5
4,34,5,14,4,360.0


Попробуем предсказывать зарплату сотрудников используя модель линейной регрессии

In [3]:
from sklearn.metrics import mean_absolute_error

X = data[['age_worker', 'projects_month', 'hours_project', 'sick_days']]
y = data['pay']
reg = LinearRegression().fit(X, y)
print(f'Weights: {reg.coef_}')
print(f'Bias: {reg.intercept_}')

pred_values = reg.predict(data[['age_worker', 'projects_month', 'hours_project', 'sick_days']])
print(f'Error: {mean_absolute_error(pred_values, y)}')

Weights: [-2.00512318e-02  1.20955527e+02  1.48303281e+01  3.34600320e+00]
Bias: -356.2489227041279
Error: 43.63739340140554


Мы видим, что ошибка равна 45.6. Это в среднем около 10% от зарплаты сотрудников, а в некоторых случаях и все 40%. Ошибка довольно таки большая, так что мы попробуем улучшить модель. 

Будем считать, что больничные никак не влияют на зарплату сотрудника, раз он не работает в эти дни.

In [4]:
X = data[['age_worker', 'projects_month', 'hours_project']]
y = data['pay']
reg = LinearRegression().fit(X, y)
print(f'Weights: {reg.coef_}')
print(f'Bias: {reg.intercept_}')

pred_values = reg.predict(data[['age_worker', 'projects_month', 'hours_project']])
print(f'Error: {mean_absolute_error(pred_values, y)}')

Weights: [-1.07940888e-02  1.21051970e+02  1.47186201e+01]
Bias: -341.1282900577841
Error: 44.16794735725151


45.7 - ошибка увеличилась! Оказывается, что больничные все же как-то влияют на зарплату, сделаем новое предположение, что зарплата не зависит от возраста сотрудника.

In [5]:
X = data[['projects_month', 'hours_project', 'sick_days']]
y = data['pay']
reg = LinearRegression().fit(X, y)
print(f'Weights: {reg.coef_}')
print(f'Bias: {reg.intercept_}')

pred_values = reg.predict(data[['projects_month', 'hours_project', 'sick_days']])
print(f'Error: {mean_absolute_error(pred_values, y)}')

Weights: [120.95682994  14.83067024   3.34375024]
Bias: -357.0242577175164
Error: 43.646612764238235


Ошибка уменьшилась, но незначительно. В таком случае мы попытаемся уменьшить ошибку за счёт объединения нескольких прихнаков. Скорее всего сотрудникам платит за время, которое онм провели за рабочими проектами. Чтобы посчитать общее время работы за месяц, мы умножим среднее количество часов потраченных на проект, на количество проектов сделанных за месяц.

In [12]:
# создаем новый признак
data['hours_total'] = data['projects_month'] * data['hours_project']
data.head()

,age_worker,projects_month,hours_project,sick_days,pay,hours_total
0,60,4,23,7,477.5,92
1,32,1,23,6,130.0,23
2,29,2,38,2,385.0,76
3,53,4,32,3,647.5,128
4,34,5,14,4,360.0,70


In [15]:
X = data[['hours_total']]
y = data['pay']
reg = LinearRegression().fit(X, y)
print(f'Weights: {reg.coef_}')
print(f'Bias: {reg.intercept_}')

pred_values = reg.predict(data[['hours_total']])
print(f'Error: {mean_absolute_error(pred_values, y)}')

Weights: [4.99272007]
Bias: 10.279893943717411
Error: 4.152037493818267


Ошибка уменьшилась в 10 раз. Мы видим, что наше предположение было правильным и зарплата в основном зависит от общего количества рабочих часов. Однако ошибка всё еще довольно большая - 4.15. Вспомним наше наблюдение, что зарплата зависит в том числе и от количества больничных и добавим данный параметр в модель линейной регрессии.  

In [14]:
X = data[['hours_total', 'sick_days']]
y = data['pay']
reg = LinearRegression().fit(X, y)
print(f'Weights: {reg.coef_}')
print(f'Bias: {reg.intercept_}')

pred_values = reg.predict(data[['hours_total', 'sick_days']])
print(f'Error: {mean_absolute_error(pred_values, y)}')

Weights: [5.  2.5]
Bias: 3.410605131648481e-13
Error: 1.573994268255774e-13


Как мы и ожидали, ошибка уменьшилась до 0 и наша модель теперь точно предсказывает зарплату сотрудника исходя из вводных данных.